## 1. PP-TinyPose模型简介
PP-TinyPose是PaddleDetecion针对移动端设备优化的实时关键点检测模型，可流畅地在移动端设备上执行多人姿态估计任务。借助PaddleDetecion自研的优秀轻量级检测模型[PicoDet](https://github.com/PaddlePaddle/PaddleDetection/blob/release/2.5/configs/picodet/README.md)，我们同时提供了特色的轻量级垂类行人检测模型.TinyPose的运行环境有以下依赖要求：

In [ ]:
PaddlePaddle>=2.2

如希望在移动端部署，则还需要：

In [ ]:
Paddle-Lite>=2.11

更多部署案例可参考[PP-TinyPose](https://github.com/PaddlePaddle/PaddleDetection/blob/release/2.5/configs/keypoint/tiny_pose/README.md)

## 2. 模型效果及应用场景
### 2.1 关键点检测任务：

#### 2.1.1 数据集：

目前KeyPoint模型支持[COCO](https://cocodataset.org/#keypoints-2017)数据集和[MPII](http://human-pose.mpi-inf.mpg.de/#overview)数据集，数据集的准备方式请参考[关键点数据准备](https://github.com/PaddlePaddle/PaddleDetection/blob/release/2.5/docs/tutorials/KeyPointConfigGuide.md)

#### 2.1.2 模型效果速览：

PP-TinyPose的检测效果为：

![](https://user-images.githubusercontent.com/15810355/181733705-d0f84232-c6a2-43dd-be70-4a3a246b8fbc.gif)




## 3. 模型如何使用

### 3.1 模型推理：
* 下载 

（不在Jupyter Notebook上运行时需要将"！"或者"%"去掉。）


In [ ]:
%cd ~/work
# 克隆PaddleDetection（从gitee上更快）,本项目以做持久化处理，不用克隆了。
!git clone https://gitee.com/paddlepaddle/PaddleDetection

* 安装

In [ ]:
# 运行脚本需在PaddleDetection目录下
%cd ~/work/PaddleDetection/

# 安装所需依赖项【已经做持久化处理，无需再安装】
!pip install pyzmq   
!pip install -r requirements.txt

# 运行脚本需在PaddleDetection目录下
%cd ~/work/PaddleDetection/
# 设置python运行目录
%env PYTHONPATH=.:$PYTHONPATH
# 设置GPU
%env CUDA_VISIBLE_DEVICES=0

# 经简单测试，提前安装所需依赖，比直接使用setup.py更快
!pip install pycocotools  
!pip install cython-bbox      
!pip install xmltodict  
!pip install terminaltables    
!pip intall motmetrics  
!pip install lap    
!pip install shapely      
!pip install pytest-benchmark    
!pip install pytest    


# 开始安装PaddleDetection 
!python setup.py install  #如果安装过程中长时间卡住，可中断后继续重新执行，

* 验证是否安装成功
如果报错，只需执行上一步操作。

In [ ]:
# 测试是否安装成功
!python ppdet/modeling/tests/test_architectures.py

* 快速体验

恭喜！ 您已经成功安装了PaddleDetection，接下来快速关键点检测效果。您可以直接下载模型库中提供的对应预测部署模型，分别获取得到行人检测模型和关键点检测模型的预测部署模型，解压即可。

In [ ]:
# 预测一张图片
python3 deploy/python/det_keypoint_unite_infer.py --det_model_dir=output_inference/picodet_s_320_pedestrian --keypoint_model_dir=output_inference/tinypose_128x96 --image_file={your image file} --device=GPU
# 预测多张图片
python3 deploy/python/det_keypoint_unite_infer.py --det_model_dir=output_inference/picodet_s_320_pedestrian --keypoint_model_dir=output_inference/tinypose_128x96 --image_dir={dir of image file} --device=GPU

# 预测一个视频
python3 deploy/python/det_keypoint_unite_infer.py --det_model_dir=output_inference/picodet_s_320_pedestrian --keypoint_model_dir=output_inference/tinypose_128x96 --video_file={your video file} --device=GPU


### 3.2 模型训练：
* 克隆PaddleDetection仓库（详见3.1）

* 数据集准备

    关键点检测模型与行人检测模型的训练集在`COCO`以外还扩充了[AI Challenger](https://arxiv.org/abs/1711.06475)数据集，各数据集关键点定义如下：
    ```
    COCO keypoint Description:
        0: "Nose",
        1: "Left Eye",
        2: "Right Eye",
        3: "Left Ear",
        4: "Right Ear",
        5: "Left Shoulder,
        6: "Right Shoulder",
        7: "Left Elbow",
        8: "Right Elbow",
        9: "Left Wrist",
        10: "Right Wrist",
        11: "Left Hip",
        12: "Right Hip",
        13: "Left Knee",
        14: "Right Knee",
        15: "Left Ankle",
        16: "Right Ankle"

    AI Challenger Description:
        0: "Right Shoulder",
        1: "Right Elbow",
        2: "Right Wrist",
        3: "Left Shoulder",
        4: "Left Elbow",
        5: "Left Wrist",
        6: "Right Hip",
        7: "Right Knee",
        8: "Right Ankle",
        9: "Left Hip",
        10: "Left Knee",
        11: "Left Ankle",
        12: "Head top",
        13: "Neck"
    ```

    由于两个数据集的关键点标注形式不同，我们将两个数据集的标注进行了对齐，仍然沿用COCO的标注形式，您可以下载[训练的参考列表](https://bj.bcebos.com/v1/paddledet/data/keypoint/aic_coco_train_cocoformat.json)并放在`dataset/`下使用。对齐两个数据集标注文件的主要处理如下：
    - `AI Challenger`关键点标注顺序调整至与COCO一致，统一是否标注/可见的标志位；
    - 舍弃了`AI Challenger`中特有的点位；将`AI Challenger`数据中`COCO`特有点位标记为未标注；
    - 重新排列了`image_id`与`annotation id`；
    利用转换为`COCO`形式的合并数据标注，执行模型训练。
    若用户需要自定义数据集，可参考[快速开始-自定义数据集](https://github.com/PaddlePaddle/PaddleDetection/tree/release/2.5/configs/keypoint#%E5%BF%AB%E9%80%9F%E5%BC%80%E5%A7%8B)

In [ ]:
# 关键点检测模型
python3 -m paddle.distributed.launch tools/train.py -c configs/keypoint/tiny_pose/tinypose_128x96.yml

# 行人检测模型
python3 -m paddle.distributed.launch tools/train.py -c configs/picodet/application/pedestrian_detection/picodet_s_320_pedestrian.yml

## 4. 方案介绍
<div align="center">
  <img src="tinypose_pipeline.png" width='800'/>
</div>